In [1]:
import os
import math
import datetime
import pandas as pd

def calculate_h_index(quotation): #h_ndex
    quotation.sort(reverse=True)  # 논문을 내림차순으로 정렬

    h_index = 0
    for i, quotation in enumerate(quotation):
        if quotation >= i + 1:
            h_index = i + 1
        else:
            break

    return h_index

def calculate_i10_index(quotation):  #i10_index
    i10_index = sum(1 for quotation in quotation if quotation >= 10)
    return i10_index

def calculate_g_index(quotation):  #g_index
    quotation.sort(reverse=True)  # 논문을 내림차순으로 정렬

    g_index = 0
    quotation_sum=0
    for i, quotation in enumerate(quotation):
        quotation_sum+=quotation
        if quotation_sum >= (i + 1)**2:
            g_index = i + 1

    return g_index

#Object 타입에서 숫자형 체크용
def is_numeric(value):
    try:
        float(value)  # float으로 변환 가능한 경우 숫자
        return True
    except (ValueError, TypeError):
        return False

def recentness(pYears):
    # dt = datetime.datetime.now()#현재날짜
    dt = 2013
    rct = 0
    lct = 0
    if len(pYears) == 0:
        rct = 0
        lct = 0
    else:
        year_avg = sum(pYears) / len(pYears)
        if year_avg >= int(dt)-2: #int(dt.year)-2:
            lct = 1
        # else:
        #     lct = max(round((1-(int(dt)-3-year_avg)*0.1),2), 0)
        elif int(dt)-15 < year_avg <= int(dt)-3:
            lct = max(round((1-(int(dt)-3-year_avg)*0.1),2), 0)

        rct += len(pYears) #연도 개수

    return rct, lct

In [9]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', None)


client = MongoClient('mongodb://10.255.92.141:27017', authSource='admin')

# keyid = 2157 #input
# idTest = client['ID']['test'] #Domestic
# publicNewfactor = client['PUBLIC']['new_factor']
# idTest_keyid = idTest.find({ 'keyId' : keyid })
# maxFactor = publicNewfactor.find_one({'keyId' : keyid})

# a = idTest.count_documents({ 'keyId' : keyid })
# print(a)



In [10]:

keyid2=2152
idTest = client['ID']['test'] #Domestic
idInter_keyid = idTest.find({ 'keyId' : keyid2 })
publicNewfactor_inter = client['PUBLIC']['new_factor_inter']
maxFactor_inter = publicNewfactor_inter.find_one({'keyId' : keyid2})

b = idTest.count_documents({ 'keyId' : keyid2 })

In [11]:
sites = ["WOS"]
name=[]
sco=[]
h_idx=[]
i10_idx=[]
g_idx=[]

df = pd.DataFrame({
    'name': name,
    'score': sco,
    'h_index': h_idx,
    'i10_index': i10_idx,
    'g_index': g_idx
})

for i in idInter_keyid:
    citations = []
    years = []
    A_id=[]
    for site in sites:
        if site in i.keys():
            papersIds = i[site]["papers"]

            for paper in papersIds:
                idSite = client[site]['Rawdata'] 
                idSite_pId = idSite.find_one({ '_id' : ObjectId(str(paper)) })

                citations.append(int(idSite_pId["citation"]))
                years.append(int(idSite_pId["issue_year"]))
            
            
            h_index = calculate_h_index(citations)
            i10_index = calculate_i10_index(citations)
            g_index = calculate_g_index(citations)
    
    A_id=i['name']
    print(A_id, citations, len(citations), years, len(years))
    lct = i['factor']['lct'] #생산성
    rct = i['factor']['rct'] #생산성
    acc = i['factor']['acc'] #주제적합도
    qual = i['factor']['interQual'] #품질
    norm_scoquality = qual/maxFactor_inter['interQual'] #정규화된 품질
    norm_recentness = ((rct/maxFactor_inter['recentness'])+lct)*0.66 #정규화된 생산성
    print(norm_scoquality, norm_recentness, acc)
    score = (norm_scoquality + norm_recentness + acc)/3*100
    # print(f'citations: {citations}, years: {years}')

    new_data={'name': A_id, 'score': score, 'h_index': h_index, 'i10_index': i10_index, 'g_index':g_index}

    df = pd.concat([df, pd.DataFrame(new_data, index=[3])], ignore_index=True)

df_score = df.sort_values(by='score', ascending=False)
df_h = df.sort_values(by='h_index', ascending=False)
df_i10 = df.sort_values(by='i10_index', ascending=False)
df_g = df.sort_values(by='g_index', ascending=False)
print(df_h)

Hua, Li [7] 1 [2018] 1
0.05107062231877884 0.22676923076923078 0.5773502691896258
Zhang, Youqiang [12, 6] 2 [2018, 2018] 2
0.10812402017047065 0.27753846153846157 0.9428090415820636
Starek, Michael [17, 10] 2 [2019, 2018] 2
0.12142517923909707 0.2885384615384616 0.7745966692414835
Kusche, Juergen [14] 1 [2018] 1
0.06181185934036625 0.22676923076923078 0.5773502691896258
Li, Ruirui [4] 1 [2018] 1
0.04263844142158996 0.22676923076923078 0.8164965809277261
Kamilov, Ulugbek S. [9] 1 [2018] 1
0.05493594977598829 0.22676923076923078 0.5773502691896258
Cao, Qunsheng [21] 1 [2018] 1
0.06818589228101045 0.22676923076923078 0.8164965809277261
Argunsah, Ali Ozgur [38] 1 [2018] 1
0.07757904265807702 0.22676923076923078 0.5773502691896258
Leng, Kaijun [34] 1 [2018] 1
0.07581271494860407 0.22676923076923078 0.5773502691896258
Nowak, E. [18] 1 [2018] 1
0.06575686181867221 0.22676923076923078 0.8164965809277261
Palenychka, M. [18] 1 [2018] 1
0.06575686181867221 0.22676923076923078 0.5773502691896258
H